In [ ]:
updateWipByDay("02","03")

In [ ]:
from datetime import date

def calculateFlowTime(startDate, endDate):
    
    fdate = date(int(startDate[0:4]),int(startDate[5:7]),int(startDate[-2:]))
    
    print(fdate)
    ldate = date(int(endDate[0:4]),int(endDate[5:7]),int(endDate[-2:]))
    print(ldate)
    delta = ldate - fdate
    print(delta.days)
    
    return(delta.days)

dateTest = "2021-03-20"

doneDateTest = "2021-05-10"

calculateFlowTime(dateTest, doneDateTest)

In [ ]:
print(wipByDay)

I am going to start splitting out the functions so they run independently

To do list:

Check whether each function and import required.
Get correct no of days in month DONE
Get correct dates for when a story has gone back to ready or backlog.
Create package
Allow input of file name

Functions to write:

- utility functions DONE
- calculate flow times DONE
- calculate wip DONE
- calculate types of work DONE
- calculate time theft


In [11]:
# Utility functions

ignoreList = ["DLK-853", "DLK-854", "DLK-893", "DLK-695"]

def validRow(row):
            
    if row[0] in ignoreList:
        #print(f"{row[0]} in ignoreList")
        #skippedStories.append(row[0])
        return False
            
    if row[inProgressColumn] == "" and row[deployingColumn] == "" and row[deployedColumn] == "" and row[doneColumn] =="": 
        #print(f"{row[0]} blank row skipped")
        #skippedStories.append(row[0])
        return False
            
        # skip if done ticket in previous month, this can happen if someone has updated commented on a done ticket
        
    if row[doneColumn] != "" and row[doneColumn][-5:-3] != currentMonth:
        #print(f"{row[0]} ticket closed in previous month, row skipped")
        #skippedStories.append(row[0])
        return False
    
    #if row[statusColumn] == "Backlog":
    #    return False
        
    return True

def convertDateFomat(jiraDate):
    year = int(jiraDate[0:4])
    month = int(jiraDate[5:7])
    day = int(jiraDate[-2:])
    
    formattedDate = date(year, month, day)
    
    return(formattedDate)

In [15]:
# Calculate Flow Times

import csv
from datetime import date
import statistics

def validRow(row):
            
    if row[0] in ignoreList:
        #print(f"{row[0]} in ignoreList")
        return False
            
    if row[inProgressColumn] == "" and row[deployingColumn] == "" and row[deployedColumn] == "" and row[doneColumn] =="": 
        #print(f"{row[0]} blank row skipped")
        return False
            
        # skip if done ticket in previous month, this can happen if someone has updated commented on a done ticket
        
    if row[doneColumn] != "" and row[doneColumn][-5:-3] != currentMonth:
        #print(f"{row[0]} ticket closed in previous month, row skipped")
        return False
    
    return True

def convertDateFomat(jiraDate):
    year = int(jiraDate[0:4])
    month = int(jiraDate[5:7])
    day = int(jiraDate[-2:])
    
    formattedDate = date(year, month, day)
    
    return(formattedDate)

def calculateFlowTime(startDate, endDate):
    
    fdate = date(int(startDate[0:4]),int(startDate[5:7]),int(startDate[-2:]))
    
    ldate = date(int(endDate[0:4]),int(endDate[5:7]),int(endDate[-2:]))

    delta = ldate - fdate

    print(f"start date {fdate} done date{ldate} flow time {delta.days}")
    
    return(delta.days)

#################################

defaultCurrentMonth = "09"

print(f"Current default month {defaultCurrentMonth}")

currentMonth = input("Enter a month in two digit form, e.g. 09: ")

if currentMonth == "" : currentMonth = defaultCurrentMonth

flowTimes = []

fileName = "/Users/richard.holloway/Development/hmrc/jira-to-analytics/dlk-output.csv"

inProgressColumn = 5
deployingColumn = 6
deployedColumn = 7
doneColumn = 8
labelsColumn = 10
createdColumn = 12
statusColumn = 13

skippedStories = []

with open(fileName, newline='') as csvfile:
    
    numberOfRows = 0
        
    for line in csvfile.readlines():
        array = line.split(',')
        numberOfRows +=1 
        
        jiraId = array[0]
        
        dateRangeForRow = []

        if array[0] == "ID": # skip header row
            continue

        if validRow(array) == False:
            skippedStories.append(array[0])
            continue
        
        if array[statusColumn] == "To Do" or array[statusColumn] == "Backlog":
            # story moved back so ignore for now.
            skippedStories.append[f"{array[0]} - story on backlog or Ready"]
            continue
        
        if array[inProgressColumn] != "":
            dateRangeForRow.append(array[inProgressColumn])
            
        if array[deployingColumn] != "":
            dateRangeForRow.append(array[deployingColumn])
            
        if array[deployedColumn] != "":
            dateRangeForRow.append(array[deployedColumn])
            
        if array[doneColumn] != "":
            dateRangeForRow.append(array[doneColumn])
        
        earliestDate = min(dateRangeForRow)
        latestDate = max(dateRangeForRow)
        
        convertedEarliestDate = convertDateFomat(earliestDate)
        convertedLatestDate = convertDateFomat(latestDate)
        
        delta = convertedLatestDate - convertedEarliestDate
        
        if array[doneColumn] != "":
            
            flowTime = delta.days
            
            flowTimes.append(flowTime)
        
            print(f"{jiraId} earliest date {earliestDate}, latest date {latestDate}, flowTime = {flowTime}, labels = {array[labelsColumn]}")
        
        else:
            
            print(f"{jiraId} earliest date {earliestDate}, latest date {latestDate}, labels = {array[labelsColumn]}")
        
print()
print(f"skippedStories {skippedStories}")

print()
print(f"average flow time = {statistics.mean(flowTimes)}")        
print(f"stdev flow time = {statistics.stdev(flowTimes)}")
print()

Current default month 09
Enter a month in two digit form, e.g. 09: 
DLK-552 earliest date 2021-03-12, latest date 2021-09-14, labels = [RawDataOnDemand;productWork]
DLK-649 earliest date 2021-03-15, latest date 2021-09-10, flowTime = 179, labels = [RequestSummary;productWork]
DLK-723 earliest date 2021-06-01, latest date 2021-06-01, labels = [RepairingDamagedData;unplannedWork]
DLK-791 earliest date 2021-06-14, latest date 2021-06-14, labels = productWork
DLK-818 earliest date 2021-06-29, latest date 2021-06-29, labels = [testLabel;unplannedWork]
DLK-833 earliest date 2021-08-17, latest date 2021-08-18, labels = [audit-simplified-observability;productWork]
DLK-845 earliest date 2021-09-01, latest date 2021-09-16, flowTime = 15, labels = productWork
DLK-851 earliest date 2021-07-30, latest date 2021-08-23, labels = unplannedWork
DLK-858 earliest date 2021-08-25, latest date 2021-09-08, flowTime = 14, labels = unplannedWork
DLK-890 earliest date 2021-08-10, latest date 2021-08-13, labels

In [47]:
# Calculate WIP

import csv
from datetime import date
import statistics
from calendar import monthrange

def updateWipByDay(earliest_date, latest_date):

    for n in range(int(earliest_date), int(latest_date)+1): #don't count last date
        
        #if n == 17: print(array[0])

        wipByDay[str(n)] +=1 

def findMaxWip(wipDictionary):
    
    maxWip = 0
    maxWipDay = ""
    
    for n in wipDictionary:
    
        if wipDictionary[n] > maxWip:
        
            maxWip = wipDictionary[n]
        
            maxWipDay = n
    
    maxWipDayAndCount = {}
    
    maxWipDayAndCount[str(maxWipDay)] = maxWip
    
    return maxWipDayAndCount

defaultCurrentMonth = "09"

print(f"Current default month {defaultCurrentMonth}")

currentMonth = input("Enter a month in two digit form, e.g. 08: ")

if currentMonth == "" : currentMonth = defaultCurrentMonth
    
numberOfDaysInMonth = monthrange(2021, int(currentMonth))[1]

fileName = "/Users/richard.holloway/Development/hmrc/jira-to-analytics/dlk-output.csv"

readyColumn = 4
inProgressColumn = 5
deployingColumn = 6
deployedColumn = 7
doneColumn = 8
labelsColumn = 10
createdColumn = 12
statusColumn = 13

skippedStories = []

wipByDay = {}

for n in range(1,numberOfDaysInMonth + 1):
    
    wipByDay[str(n)] = 0

with open(fileName, newline='') as csvfile:
    
    numberOfRows = 0
        
    for line in csvfile.readlines():
        array = line.split(',')
        numberOfRows +=1 
        
        jiraId = array[0]
        
        dateRangeForRow = []

        if array[0] == "ID": # skip header row
            continue
            
        ###########

        if validRow(array) == False:
            skippedStories.append(array[0])
            continue
                
        if array[statusColumn].find("To Do") >=0 :            # story moved back so ignore for now.
            skippedStories.append(f"{array[0]} - story on backlog or Ready")
            continue        

        if array[inProgressColumn] != "":
            dateRangeForRow.append(array[inProgressColumn])
            
        if array[deployingColumn] != "":
            dateRangeForRow.append(array[deployingColumn])
            
        if array[deployedColumn] != "":
            dateRangeForRow.append(array[deployedColumn])
            
        if array[doneColumn] != "":
            dateRangeForRow.append(array[doneColumn])
        
        earliestDate = min(dateRangeForRow)
        latestDate = max(dateRangeForRow)
        
        convertedEarliestDate = convertDateFomat(earliestDate)
        convertedLatestDate = convertDateFomat(latestDate)
        
        if convertedEarliestDate.month != int(currentMonth):
            wipEarliestDate = "01"
        
        else:
            wipEarliestDate = str(convertedEarliestDate.day)
        
        # if not date in the done column, the story is still open or back on Ready.  Need to figoure out backlog date
        
        if array[doneColumn] == "":
            latestWipDate = numberOfDaysInMonth
        
        else:
            latestWipDate = str(convertedLatestDate.day)
            
        #cover the case where a story might have been moved back
        
        if array[readyColumn] > latestDate:
            latest
        
        updateWipByDay(wipEarliestDate, latestWipDate)
            
        #print(f"e wip day {wipEarliestDate} l wip day {latestWipDate}")
        

wip = findMaxWip(wipByDay)

print(f"Max wip = {wip}")

print(f"average wip = {statistics.mean(wipByDay.values())}")

#print(skippedStories)

Current default month 09
Enter a month in two digit form, e.g. 08: 
Max wip = {'2': 12}
average wip = 9.833333333333334


DLK-552 Y
DLK-695 N NOT ON EXCLUDE LIST
DLK-723 Y
DLK-791 IN READY AMI
DLK-818 Y
DLK-833 Y
DLK-851 Y
DLK-890 Y
DLK-897 Y
DLK-906 Y
DLK-908 IN READY HUDI
DLK-925 Y

In [21]:
# calculate type of work

import csv
from datetime import date
import statistics

def updateTypeOfWorkCount():
    pass

defaultCurrentMonth = "09"

print(f"Current default month {defaultCurrentMonth}")

currentMonth = input("Enter a month in two digit form, e.g. 08: ")

if currentMonth == "" : currentMonth = defaultCurrentMonth

fileName = "/Users/richard.holloway/Development/hmrc/jira-to-analytics/dlk-output.csv"

inProgressColumn = 5
deployingColumn = 6
deployedColumn = 7
doneColumn = 8
labelsColumn = 10
createdColumn = 12
statusColumn = 13

noOfStoriesTouched = 0

unplannedWorkCount = 0
productWorkCount = 0
teamWorkCount = 0


unplannedWork = "unplannedWork"
productWork = "productWork"
teamWork = "teamWork"

skippedStories = []

with open(fileName, newline='') as csvfile:
    
    numberOfRows = 0
        
    for line in csvfile.readlines():
        array = line.split(',')
        numberOfRows +=1 
        
        jiraId = array[0]
        
        dateRangeForRow = []

        if array[0] == "ID": # skip header row
            continue

        if validRow(array) == False:
            skippedStories.append(array[0])
            continue
        
        noOfStoriesTouched += 1
         
        if array[labelsColumn].find(unplannedWork) >=0 : 
            unplannedWorkCount += 1
        if array[labelsColumn].find(productWork) >= 0 : 
            productWorkCount += 1
        if array[labelsColumn].find(teamWork) >= 0 : 
            teamWorkCount += 1
            
print(f"no of stories touched {noOfStoriesTouched} unplannedWork {unplannedWorkCount} productWork {productWorkCount} teamWork {teamWorkCount}")

Current default month 09
Enter a month in two digit form, e.g. 08: 
no of stories touched 16 unplannedWork 7 productWork 6 teamWork 3
